# 영화 댓글 감성 분석 (한글 처리)
- 네이버 영화에서 특정 영화에 대한 댓글 20만건을 크롤링한 데이터를 분석한다. 댓글에 적용된 별점을 기준으로 3점 이상은 긍정, 2점 이하는 부정으로 분류하였다.

## 1. 작업 준비

### 1) 형태소 분석엔진 설치

In [ ]:
# !git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
# !bash Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab_light_220429.sh

### 패키지 참조

In [2]:
import sys
sys.path.append('../../')
import helper

In [3]:
import numpy as np
import seaborn as sb
import requests

from pandas import DataFrame
from pandas import read_excel
from konlpy.tag import Mecab
from matplotlib import pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

## 2. 데이터셋 준비하기
### 1) 리뷰 데이터셋

In [ ]:
origin = read_excel("https://data.hossam.kr/F03/movie_review.xlsx")
origin.head()

,label,document
0,부정,아 더빙.. 진짜 짜증나네요 목소리
1,긍정,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2,부정,너무재밓었다그래서보는것을추천한다
3,부정,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4,긍정,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...


### 2) 불용어 데이터셋
#### 데이터셋 로드

In [5]:
r = requests.get("https://data.hossam.kr/korean_stopwords.txt")
r.encoding = 'utf-8'
print(len(r.text))

2972


#### 불용어만 리스트로 추출

In [6]:
stopwords = r.text.split("\n")
print(stopwords)

['이', '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하', '때문', '그것', '두', '말하', '알', '그러나', '받', '못하', '일', '그런', '또', '문제', '더', '사회', '많', '그리고', '좋', '크', '따르', '중', '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집', '살', '모르', '적', '월', '데', '자신', '안', '어떤', '내', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', '다른', '어떻', '여자', '개', '전', '들', '사실', '이렇', '점', '싶', '말', '정도', '좀', '원', '잘', '통하', '소리', '놓', '!', '"', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '...', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '<', '=', '>', '?', '@', '\\', '^', '_', '`', '|', '~', '·', '—', '——', '‘', '’', '“', '”', '…', '、', '。', '〈', '〉', '《', '》', '가', '가까스로', '가령', '각', '각각', '각자', '각종', '갖고말하자면', '같다', '같이', '개의치않고', '거니와', '거바', '거의', '것', '것과 같이', '것들', '게다가', '게우다', '겨우', '견지에서', '결과에 이르다', '결국', '결론을 낼 수 있다', '겸사겸사', '고려하면', '고로', '곧', '공

## 3. 데이터 전처리
### 1) 리뷰글에서 한글을 제외한 `영어, 특수문자` 제거

In [7]:
df = origin.copy()

# 한글을 제외한 나머지 글자들을 빈 문자열로 대체
df['document'] = df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)

# document 컬럼의 데이터들 중에서 빈 문자열만 존재하는 항목은 결측치로 대체
df['document'].replace('', np.nan, inplace=True)

# 전체 데이터 셋 크기 확인
print('데이터 크기: ', df['document'].shape)

# 결측치 확인
print('결측치 크기: ', df['document'].isna().sum())

데이터 크기:  (200000,)
결측치 크기:  1091


### 2) 결측치 정제

In [8]:
df.dropna(inplace=True)
df.isna().sum()

label       0
document    0
dtype: int64

### 3) 종속변수 라벨링

In [9]:
df['label'] = df['label'].replace({'긍정': 1, '부정': 0})
df['label'].value_counts()

label
1    99511
0    99398
Name: count, dtype: int64

### 4) 리뷰글에 대한 형태소 분석

In [11]:
if sys.platform == 'win32':
    mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")
else:
    mecab = Mecab()

# 문장내 형태소들을 저장할 리스트
word_set = []

# 덧글 내용에 대해 반복 처리
for i, v in enumerate(df['document']):
    # 덧글 하나에 대한 형태소 분석
    morphs = mecab.morphs(v)
    # print(morphs)
    # if i > 5:
    #     break

    # 형태소 분석 결과에서 불용어를 제외한 단어만 별도의 리스트로 생성
    confirm_words = []
    for j in morphs:
        if j not in stopwords:
            confirm_words.append(j)

    # 불용어를 제외한 형태소 리스트를 통째로 word_set에 저장함
    # -> word_set은 2차원 리스트가 된다. 1차원이 덧글 단위임
    word_set.append(confirm_words)

# 상위 3건만 출력해서 확인
word_set[:3]

[['빙', '진짜', '짜증', '네요', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '가볍', '구나'],
 ['너무', '재', '밓었다그래서보는것을추천한다']]

### 5) 형태소 토큰화
#### 전체 단어에 대한 토큰화

In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(word_set)
print(f'전체 단어수: {len(tokenizer.word_index)}')

전체 단어수: 56217


#### 각 단어별 빈도수 확인
- `(단어: 빈도수)` 형태의 튜플을 원소로 갖는 리스트를 반환

In [ ]:
tokenizer.word_counts.items()

#### 3회 이상 자주 등장하는 단어의 수 구하기

In [13]:
# 사용 빈도가 높다고 판단할 등장 회수
threshold = 3

# 전체 단어의 수
total_cnt = len(tokenizer.word_index)

# 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트할 값
rare_cnt = 0

# 훈련 데이터의 전체 단어 빈도수 총 합
total_freq = 0

# 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합
rare_freq = 0

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 미만인 희귀 단어의 수: %s' % (threshold, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

# 자주 등장하는 단어 집합의 크기 구하기 -> 이 값이 첫 번째 학습층의 input 수가 된다.
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :', vocab_size)

단어 집합(vocabulary)의 크기 : 56217
등장 빈도가 3번 미만인 희귀 단어의 수: 31410
단어 집합에서 희귀 단어의 비율: 55.87277869683548
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.7947824710391664
단어 집합의 크기 : 24809


#### 자주 등장하는 단어를 제외한 나머지 단어를 OOV로 처리하여 최종 토큰화 진행

In [14]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV')
tokenizer.fit_on_texts(word_set)
token_set = tokenizer.texts_to_sequences(word_set)
print('토큰의 크기 :', len(token_set))

토큰의 크기 : 198909


In [15]:
token_set

[[746, 18, 144, 17, 554],
 [865, 379, 391, 500, 3, 66, 1300, 26, 806, 274],
 [12, 126, 1],
 [6149, 92, 3864, 203, 48, 2, 4, 28, 3181],
 [1,
  7798,
  883,
  26,
  2500,
  29,
  3,
  2531,
  972,
  20,
  10,
  19,
  29,
  14232,
  17842,
  303,
  6,
  2675,
  1544,
  4],
 [523, 1, 17843, 326, 1338, 536, 1456, 527, 25, 1, 3, 33, 2065, 6, 592],
 [172, 260, 275, 1771, 651, 4],
 [165,
  1125,
  6,
  81,
  4,
  302,
  585,
  12045,
  17844,
  26,
  1244,
  168,
  14,
  696,
  240,
  401,
  359,
  449,
  4,
  3060,
  8499,
  10,
  1219,
  1219,
  37,
  2,
  206,
  6,
  4,
  26,
  58,
  2,
  10,
  1],
 [71, 11, 6, 48, 2, 2, 3],
 [479, 28, 150, 7, 568, 443, 42, 10, 1245, 468, 543, 40, 10, 12, 15749],
 [1],
 [42,
  314,
  2116,
  59,
  134,
  32,
  4,
  115,
  2252,
  710,
  12996,
  2,
  522,
  2501,
  1959,
  5815,
  4],
 [190, 62, 15750, 5, 1, 6978, 1, 20580, 308, 3809, 26, 1036, 8, 1484],
 [2268, 5, 1103, 65, 4, 8500, 2481, 10, 43, 23, 685, 1, 6, 9, 74],
 [736,
  7,
  3517,
  17845,
  18,
  

#### 토큰화 결과의 길이가 0인 항목 찾기

In [16]:
# 토큰화 결과 길이가 0인 항목의 index 찾기
drop_target_index = []

for i, v in enumerate(token_set):
    if len(v) < 1:
        drop_target_index.append(i)

print("길이가 0인 항목의 수: ", len(drop_target_index))

print("길이가 0인 항목의 인덱스 모음: ", drop_target_index)

길이가 0인 항목의 수:  985
길이가 0인 항목의 인덱스 모음:  [151, 159, 404, 412, 470, 795, 1307, 1480, 1544, 2206, 2220, 2287, 2365, 2572, 3290, 3498, 3549, 4332, 4417, 4712, 4875, 4950, 5003, 5090, 5110, 5195, 5335, 5745, 5843, 5847, 6224, 6371, 6430, 7095, 7221, 7389, 7544, 7594, 8145, 8595, 8751, 9205, 9208, 9476, 9539, 9586, 9630, 9989, 10000, 10091, 10675, 10713, 10741, 11258, 11711, 11988, 12100, 12467, 12484, 13159, 13579, 13727, 13859, 13871, 14055, 14306, 14390, 14428, 14675, 14812, 15360, 15714, 15715, 16029, 16202, 16504, 17232, 17596, 17713, 17976, 18116, 18715, 19039, 19418, 19534, 19637, 19868, 20342, 20676, 20768, 20815, 20866, 21064, 21111, 21624, 22032, 22271, 22410, 22453, 22470, 22566, 22729, 22739, 22745, 22854, 23061, 23078, 23098, 23277, 23329, 23362, 23557, 23710, 24205, 24695, 24892, 25118, 26087, 26124, 26145, 26150, 26158, 26435, 26913, 27592, 28075, 28133, 28311, 28917, 28943, 29268, 29323, 29620, 30002, 30015, 30016, 30550, 30580, 30607, 30867, 30883, 30960, 30973, 31390, 31543,

#### 토큰화 결과의 길이가 0인 항목 삭제하기

In [22]:
token_set2 = np.asarray(token_set, dtype="object")

In [23]:
# 토큰 결과에서 해당 위치의 항목들을 삭제한다.
fill_token_set = np.delete(token_set2, drop_target_index, axis=0)

# 종속변수에서도 같은 위치의 항목들을 삭제해야 한다.
label_set = np.delete(df['label'].values, drop_target_index, axis=0)

print("독립변수(덧글) 데이터 수: ", len(fill_token_set))
print("종속변수(레이블) 데이터 수: ", len(label_set))

독립변수(덧글) 데이터 수:  197924
종속변수(레이블) 데이터 수:  197924


## 4. 탐색적 데이터 분석
### 1) 각 문장별로 몇 개의 단어를 포함하고 있는지 측정

In [ ]:
word_counts = []

for s in fill_token_set:
    word_counts.append(len(s))

print(word_counts)

### 2) 리뷰 중 가장 많은 단어를 사용한 리뷰와 가장 적은 단어를 사용한 리뷰의 단어 수

In [ ]:
max_word_count = max(word_counts)
min_word_count = min(word_counts)
print('리뷰의 최대 단어수 :',max_word_count)
print('리뷰의 최소 단어수 :',min_word_count)

### 3) 히스토그램으로 단어 분포 수 확인

In [ ]:
# 히스토그램의 범위 산정
hist_values, hist_bins = np.histogram(word_counts, range=(0, max_word_count), bins=7)
hist_bins = hist_bins.astype(np.int64)

plt.figure(figsize=(10, 5))
sb.histplot(word_counts, bins=7, binrange=(0, max_word_count))
plt.xticks(hist_bins, hist_bins)

for i, v in enumerate(hist_values):
    x = hist_bins[i] + ((hist_bins[i+1] - hist_bins[i]) / 2)
    plt.text(x=x, y=v, s=str(v), fontsize=12, verticalalignment='bottom', horizontalalignment='center')

plt.show()
plt.close()

## 5. 데이터셋 분할하기
### 1) 랜덤 시드 고정

In [ ]:
np.random.seed(777)

### 2) 패딩 처리
- 최대 단어수를 갖고 있는 문장을 기준으로 그보다 적은 단어를 갖고 잇는 문장은 최대 단어수와 동일해 질 때까지 0으로 채워 넣는다

In [ ]:
max_word_count = max(word_counts)
pad_token_set = pad_sequences(fill_token_set, maxlen=max_word_count)
pad_token_set

### 3) 데이터 분할

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(pad_token_set, label_set, test_size = 0.3, random_state = 777)
print("훈련용 데이터셋 크기: %s, 검증용 데이터셋 크기: %s" % (x_train.shape,  x_test.shape))
print("훈련용 레이블 크기: %s, 검증용 레이블 크기: %s" % (y_train.shape,  y_test.shape))

## 6. 모델 개발
### 1) 모델 정의

In [ ]:
#모델 개발
my_model = Sequential()

# input_dim의 크기는 토큰 생성시 지정한 최대 단어수(vocab_size)와 동일하게 설정
# output_dim의 크기는 input_dim보다 작은 값 중에서 설정
my_model.add(Embedding(input_dim = vocab_size, output_dim = 32))
my_model.add(GRU(128))
my_model.add(Dense(1, activation='sigmoid'))
my_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(my_model.summary())

### 2) 학습하기

In [ ]:
result = my_model.fit(x_train, y_train, batch_size = 10, epochs = 500, validation_data=(x_test, y_test), callbacks = [
    EarlyStopping(monitor = 'val_loss', patience=5, verbose = 1),
    ReduceLROnPlateau(monitor= "val_loss", patience=3, factor = 0.5, min_lr=0.0001, verbose=1)
])

## 7. 학습결과 평가

In [ ]:
# helper.tf_result_plot(result)

evaluate1 = my_model.evaluate(x_train, y_train)
print("최종 훈련 손실률: %f, 최종 훈련 정확도: %f" % (evaluate1[0], evaluate1[1]))

evaluate2 = my_model.evaluate(x_test, y_test)
print("최종 검증 손실률: %f, 최종 검증 정확도: %f" % (evaluate2[0], evaluate2[1]))

In [ ]:
# 학습 결과
result_df = DataFrame(result.history)
result_df['epochs'] = result_df.index+1
result_df.set_index('epochs', inplace = True)
result_df

In [ ]:
# 학습 결과 시각화
# 그래프 기본설정
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False

# 그래프를 그리기 위한 객체 생성
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5), dpi=150)

# 1) 훈련 및 검증 손실 그리기
sb.lineplot(x=result_df.index,
            y='loss',
            data=result_df,
            color='lightblue',
            label='훈련 손실률',
            ax=ax1)
sb.lineplot(x=result_df.index,
            y='val_loss',
            data=result_df,
            color='lightcoral',
            label='검증 손실률',
            ax = ax1)
ax1.set_title('훈련 및 검증 손실률')
ax1.set_xlabel('반복회차')
ax1.set_ylabel('손실률')
ax1.grid()
ax1.legend()

# 2) 훈련 및 검증 정확도 그리기
sb.lineplot(x=result_df.index,
            y='acc',
            data=result_df,
            color='lightblue',
            label='훈련 정확도',
            ax=ax2)
sb.lineplot(x=result_df.index,
            y='val_acc',
            data=result_df,
            color='lightcoral',
            label='검증 정확도',
            ax=ax2)
ax2.set_title('훈련 및 검증 정확도')
ax2.set_xlabel('반복회차')
ax2.set_ylabel('정확도')
ax2.grid()
ax2.legend()

plt.show()
plt.close()

## 7. 학습 결과 적용
### 1) 훈련 데이터에 대한 예측 결과 산정

In [ ]:
result = my_model.predict(x_train, batch_size=10)
data_count, case_count = result.shape
print("%d개의 검증 데이터가 %d개의 경우의 수를 갖는다." % (data_count, case_count))
result

### 2) 예측 결과를 1차원 배열로 변환

In [ ]:
f_results = result.flatten()
f_results

### 3)  학습 결과 확인

In [ ]:
kdf = DataFrame({
    'train': y_train,
    'pred' : np.round(f_results)
})

kdf['pred'] = kdf['pred'].astype('int')

cm = confusion_matrix(kdf['train'], kdf['pred'])

plt.figure(figsize=(7, 3))
sb.heatmap(cm, annot = True, fmt = 'd',cmap = 'Blues')
plt.show()
plt.close()

## 8. 임의의 문장에 대한 분류

In [ ]:
# 임의의 문장
comment = [
        "오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 재미있는 영화를 만났습니다 강렬하게 유쾌함",
        "시간 때우기에도 아까운 영화",
        "시나리오에 아까운 캐스팅. 시간만 아까웠다.",
        "강력 추천~!!!",
        "감독의 유머감각이 나와 잘 맞는듯 하다. 유쾌하게 봤다."
]

# 형태소 분석 엔진
mecab = Mecab()

# 불용어를 제거한 형태소들이 저장될 리스트
word_set = []

# 문장 수 만큼 반복
for c in comment:
    # 형태소 분석
    morphs = mecab.morphs(c)
    # 불용어 제거
    tmp_word = []
    for j in morphs:
        if j not in stopwords:
            tmp_word.append(j)

    word_set.append(tmp_word)

# 자주 등장하는 단어를 제외한 나머지 단어를 OOV로 처리하여 최종 토큰화 수행
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV')
tokenizer.fit_on_texts(word_set)
token_set = tokenizer.texts_to_sequences(word_set)

# 최대 길이에 맞춰서 패딩 처리
pad_token_set = pad_sequences(token_set, maxlen=max_word_count)

# 전처리가 완료된 말뭉치를 학습모델에 적용하여 예측하기
result = my_model.predict(pad_token_set)

# 결과 분석
for i, v in enumerate(result.flatten()):
    p = np.round(v * 100, 1)

    if p > 50:
        print("%.1f%%의 확률로 긍정입니다. >> %s" % (p, comment[i]))
    else:
        print("%.1f%%의 확률로 부정입니다. >> %s" % (100-p, comment[i]))